In [87]:
import pandas as pd
from pathlib import Path
import requests
from functools import wraps
from functools import reduce

In [88]:
def check_cache(f):
    @wraps(f)
    def wrapper(src:str, dst:str, min_size=100) -> Path:
        url = Path(src) 
        fn  = url.name  
        dsn = Path(f"{dst}/{fn}") 
        if dsn.is_file() and dsn.stat().st_size > min_size:
            print(f"+ {dsn} found locally!")
            return(dsn)
        else:
            print(f"+ {dsn} not found, downloading!")
            return(f(src, dsn))
    return wrapper

@check_cache
def cache_data(src:Path, dst:Path) -> str:
    """Downloads a remote file.
    
    
        
    Returns
    -------
    str
        A string representing the local location of the file.
    """
      
    # Create...
    if not dst.parent.exists():
        dst.parent.mkdir(parents=True, exist_ok=True)
        
    # Download and write the file
    with dst.open(mode='wb') as file:
        response = get(src)
        file.write(response.content)
        
    print(' + Done downloading...')

    return dst.resolve()

In [89]:
ddir = Path('data')
spath = 'https://raw.githubusercontent.com/yxing82/census-data/refs/heads/main/'
df_asw = pd.read_csv(cache_data(spath + 'ASW.csv',
                              ddir), skiprows=8)
df_mr1 = pd.read_csv(cache_data(spath + 'MR1-ward-results.csv',
                              ddir))
df_rgc = pd.read_csv(cache_data(spath + 'Ward_to_Local_Authority_District_to_County_to_Region_to_Country_(December_2017)_Lookup_in_United_Kingdom_version_2.csv',
                              ddir))

print('Done.')

+ data/ASW.csv found locally!
+ data/MR1-ward-results.csv found locally!
+ data/Ward_to_Local_Authority_District_to_County_to_Region_to_Country_(December_2017)_Lookup_in_United_Kingdom_version_2.csv found locally!
Done.


In [90]:
ddir_2 = Path('data/mr2')
spath_2 = "https://raw.githubusercontent.com/yxing82/census-data/refs/heads/main/mr2/"
df_mr2_deprivation = pd.read_csv(cache_data(spath_2 + '2151161339385962_deprivation.csv',
                              ddir_2), skiprows=8)
df_mr2_agemean = pd.read_csv(cache_data(spath_2 + '2940642151781712_age_mean.csv',
                              ddir_2), skiprows=8)
df_mr2_economicactivity = pd.read_csv(cache_data(spath_2 + 'mr2_economic_activity.csv',
                              ddir_2), skiprows=9)
df_mr2_ethnic = pd.read_csv(cache_data(spath_2 + 'mr2_ehtnic.csv',
                              ddir_2), skiprows=8)
df_mr2_tenture = pd.read_csv(cache_data(spath_2 + 'mr2_housing_tenture.csv',
                              ddir_2), skiprows=8)
df_mr2_occup = pd.read_csv(cache_data(spath_2 + 'mr2_occupation.csv',
                              ddir_2), skiprows=9)
df_mr2_qualif = pd.read_csv(cache_data(spath_2 + 'mr2_qualification.csv',
                              ddir_2), skiprows=8)
df_mr2_socigrade = pd.read_csv(cache_data(spath_2 + 'mr2_social_grade.csv',
                              ddir_2), skiprows=8)

df_mr2_households = pd.read_csv(cache_data(spath_2 + 'mr2_household.csv',
                              ddir_2), skiprows=8)
df_mr2_resident = pd.read_csv(cache_data(spath_2 + 'mr2_reisdent.csv',
                              ddir_2), skiprows=8)

+ data/mr2/2151161339385962_deprivation.csv found locally!
+ data/mr2/2940642151781712_age_mean.csv found locally!
+ data/mr2/mr2_economic_activity.csv found locally!
+ data/mr2/mr2_ehtnic.csv found locally!
+ data/mr2/mr2_housing_tenture.csv found locally!
+ data/mr2/mr2_occupation.csv found locally!
+ data/mr2/mr2_qualification.csv found locally!
+ data/mr2/mr2_social_grade.csv found locally!
+ data/mr2/mr2_household.csv found locally!
+ data/mr2/mr2_reisdent.csv found locally!


/tmp/ipykernel_82/4162486983.py:5: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df_mr2_agemean = pd.read_csv(cache_data(spath_2 + '2940642151781712_age_mean.csv',


## Data Processing

### Process MR2 dfs

In [91]:
# pre-process deprived df - deprived and multideprived

# List of columns you need to fix
cols_to_fix = [
    'All categories: Classification of household deprivation',
    'Household is not deprived in any dimension',
    'Household is deprived in 1 dimension',
    'Household is deprived in 2 dimensions',
    'Household is deprived in 3 dimensions',
    'Household is deprived in 4 dimensions'
]

# 1. Remove commas (common culprit) and convert to numeric
for col in cols_to_fix:
    # Force convert to string first, replace comma, then convert to number
    # errors='coerce' turns "Bad Data" into NaN instead of crashing the script
    df_mr2_deprivation[col] = pd.to_numeric(
        df_mr2_deprivation[col].astype(str).str.replace(',', ''), 
        errors='coerce'
    )
# Check the types now
print(df_mr2_deprivation.dtypes)

# Shorten the long column names into variables for easier reading
col_total = 'All categories: Classification of household deprivation'
col_not_deprived = 'Household is not deprived in any dimension'
col_1 = 'Household is deprived in 1 dimension'
col_2 = 'Household is deprived in 2 dimensions'
col_3 = 'Household is deprived in 3 dimensions'
col_4 = 'Household is deprived in 4 dimensions'

# 1. Calculate numerator for "Deprived" (At least 1 dimension)
df_mr2_deprivation['numer_deprived'] = df_mr2_deprivation[col_total] - df_mr2_deprivation[col_not_deprived]

# 2. Calculate numerator for "MultiDepriv" (At least 2 dimensions)
df_mr2_deprivation['numer_multidepriv'] = (
    df_mr2_deprivation[col_2] + 
    df_mr2_deprivation[col_3] + 
    df_mr2_deprivation[col_4]
)

2011 ward                                                   object
All categories: Classification of household deprivation    float64
Household is not deprived in any dimension                 float64
Household is deprived in 1 dimension                       float64
Household is deprived in 2 dimensions                      float64
Household is deprived in 3 dimensions                      float64
Household is deprived in 4 dimensions                      float64
dtype: object


In [92]:
# use a dictionary to loop over dfs to calculate the pct_columns

# 1. Give each DataFrame a meaningful name in a dictionary
# Instead of a list [df1, df2], we use keys.
dfs_dict = {
    'ethnicity': df_mr2_ethnic,
    'eco': df_mr2_economicactivity,   
    'housing': df_mr2_tenture,
    'dep': df_mr2_deprivation,
    'occup': df_mr2_occup,
    'qualif': df_mr2_qualif,
    'socigrade': df_mr2_socigrade
    
}

In [93]:
# 2. Create the "Mapping Dictionary"
# This tells Python: "When you see 'ethnicity', use these columns."
config_map = {
    'ethnicity': {
        'denom': 'All usual residents',
        'numerators': {
            'White': 'White', 
            'Black/African/Caribbean/Black British': 'Black',
            'Asian/Asian British': 'Asian'
        }
    },
    'eco': {
        'denom': 'All usual residents aged 16 to 74',  
        'numerators': {
            'Economically active': 'Unemp', 
            'Economically active: Unemployed': 'UnempRate_EA',
            'Economically inactive: Student (including full-time students)': 'Students'
        }
    },
    'housing': {
        'denom': 'All households', 
        'numerators': {
            'Owned': 'Owned', 
            'Owned: Owned outright': 'OwnedOutright',
            'Social rented': 'SocialRent',
            'Private rented': 'PrivateRent'
        }
    },
    'dep': {
        'denom': 'All categories: Classification of household deprivation', 
        'numerators': {
            'numer_deprived': 'Deprived', 
            'numer_multidepriv': 'MultiDepriv'
        }
    },
    'occup': {
        'denom': 'All categories: NS-SeC', 
        'numerators': {
            '1. Higher managerial, administrative and professional occupations': 'HigherOccup', 
            '7. Routine occupations': 'RoutineOccup',
            '8. Never worked and long-term unemployed': 'LTU'
        }
    },
    'qualif': {
        'denom': 'All categories: Highest level of qualification', 
        'numerators': {
            'No qualifications': 'NoQuals', 
            'Highest level of qualification: Level 1 qualifications': 'L1Quals',
            'Highest level of qualification: Level 4 qualifications and above': 'L4Quals_plus'
        }
    },
    'socigrade': {
        'denom': 'Approximated social grade', 
        'numerators': {
            'numer_C1C2DE': 'C1C2DE', 
            'numer_C2DE': 'C2DE',  
            'numer_DE': 'DE'
        }
    },
}

In [94]:
id_col = '2011 ward'
processed_dfs = []

for name, df in dfs_dict.items():
    print(f"Processing {name}...")

    # --- STEP A: CLEANING ---
    # Convert "1,200" (string) to 1200.0 (float)
    for col in df.columns:
        if col != id_col:
            df[col] = pd.to_numeric(df[col].astype(str).str.replace(',', ''), errors='coerce')

    # --- STEP B: CUSTOM MATH ---
    if name == 'deprivation':
        # ... (your existing deprivation logic) ...
        pass # placeholder

    elif name == 'socigrade':
        # define short variable names for readability
        col_c1 = 'Approximated social grade C1'
        col_c2 = 'Approximated social grade C2'
        col_de = 'Approximated social grade DE'
        
        # 1. Calculate C1 + C2 + DE
        df['numer_C1C2DE'] = df[col_c1] + df[col_c2] + df[col_de]
        
        # 2. Calculate C2 + DE
        df['numer_C2DE'] = df[col_c2] + df[col_de]
        
        # 3. Create a clean copy for DE (optional, but keeps things uniform)
        df['numer_DE'] = df[col_de]

    # --- STEP C: CALCULATE % ---
    if name in config_map:
        specs = config_map[name]
        denom_col = specs['denom']
        numerator_map = specs['numerators'] # { 'Original': 'CleanName' }

        # Get columns (keys)
        target_cols = list(numerator_map.keys())

        # Calculate %
        pct_data = df[target_cols].div(df[denom_col], axis=0) * 100

        # Rename to clean names
        pct_data = pct_data.rename(columns=numerator_map)

        # Select Final Columns (Ward + Clean Percentages)
        result = pd.concat([df[[id_col]], pct_data], axis=1)
        
        processed_dfs.append(result)

Processing ethnicity...
Processing eco...
Processing housing...
Processing dep...
Processing occup...
Processing qualif...
Processing socigrade...


In [95]:
df_mr2 = reduce(lambda left, right: pd.merge(left, right, on='2011 ward', how='outer'), 
                  processed_dfs)

df_mr2.rename(columns={id_col: 'ward_code'}, inplace=True)

df_mr2.head(2)

,ward_code,White,Black,Asian,Unemp,UnempRate_EA,Students,Owned,OwnedOutright,SocialRent,...,MultiDepriv,HigherOccup,RoutineOccup,LTU,NoQuals,L1Quals,L4Quals_plus,C1C2DE,C2DE,DE
0,E05000001 : Aldersgate,84.505119,0.750853,8.737201,78.951679,2.784603,3.685504,60.844749,40.525114,4.680365,...,8.447489,43.407043,0.655201,1.719902,1.851852,2.592593,77.555556,27.917282,3.692762,1.772526
1,E05000005 : Bishopsgate,69.369369,1.801802,21.621622,84.186047,1.860465,6.511628,17.730496,11.347518,2.836879,...,9.219858,30.232558,0.930233,1.860465,4.166667,3.240741,65.277778,46.762590,11.510791,5.035971


In [96]:
df_mr2_households = df_mr2_households.rename(columns={
    '2011 ward': 'ward_code', 
    '2011': 'Households'   
})

df_mr2_resident = df_mr2_resident.rename(columns={
    '2011 ward': 'ward_code', 
    '2011': 'Residents'   
})

In [97]:
df_mr2 = pd.merge(df_mr2, df_mr2_households, on='ward_code', how='left')
df_mr2 = pd.merge(df_mr2, df_mr2_resident, on='ward_code', how='left')

In [98]:
df_mr2.head(2)

,ward_code,White,Black,Asian,Unemp,UnempRate_EA,Students,Owned,OwnedOutright,SocialRent,...,RoutineOccup,LTU,NoQuals,L1Quals,L4Quals_plus,C1C2DE,C2DE,DE,Households,Residents
0,E05000001 : Aldersgate,84.505119,0.750853,8.737201,78.951679,2.784603,3.685504,60.844749,40.525114,4.680365,...,0.655201,1.719902,1.851852,2.592593,77.555556,27.917282,3.692762,1.772526,876,"1,465"
1,E05000005 : Bishopsgate,69.369369,1.801802,21.621622,84.186047,1.860465,6.511628,17.730496,11.347518,2.836879,...,0.930233,1.860465,4.166667,3.240741,65.277778,46.762590,11.510791,5.035971,141,222


### Process ASW dfs

In [99]:
df_asw.head(2)

,2011 ward,All usual residents,Age 0 to 4,Age 5 to 7,Age 8 to 9,Age 10 to 14,Age 15,Age 16 to 17,Age 18 to 19,Age 20 to 24,Age 25 to 29,Age 30 to 44,Age 45 to 59,Age 60 to 64,Age 65 to 74,Age 75 to 84,Age 85 to 89,Age 90 and over
0,E05001553 : Bank Top,4032,250.0,116.0,63.0,219.0,42.0,99.0,108.0,282.0,338.0,766.0,791.0,246.0,323.0,255.0,71.0,63.0
1,E05001554 : Central,4104,251.0,111.0,60.0,205.0,40.0,91.0,160.0,399.0,360.0,842.0,727.0,208.0,336.0,236.0,52.0,26.0


In [100]:
# 1. CLEANING
id_col = '2011 ward' 

for col in df_asw.columns:
    if col != id_col:
        # Ensure numbers are clean
        df_asw[col] = pd.to_numeric(df_asw[col].astype(str).str.replace(',', ''), errors='coerce')

# 2. CALCULATION LOOP
denominator = df_asw['All usual residents']

# Create a list to track the new column names you are creating
final_age_cols = []

for col in df_asw.columns:
    if col.startswith('Age'):
        # --- NAME FORMATTING LOGIC ---
        # 1. Replace "Age " with "Age_" (note the space) -> "Age_0 to 4"
        # 2. Remove all remaining spaces -> "Age_0to4"
        new_col_name = col.replace('Age ', 'Age_').replace(' ', '')
        
        # Calculate the percentage
        df_asw[new_col_name] = (df_asw[col] / denominator) * 100
        
        # Add this new name to our list so we can keep it later
        final_age_cols.append(new_col_name)

# 3. SELECT FINAL COLUMNS
# Keep the Ward ID + the specific new columns we just created
keep_cols = [id_col] + final_age_cols

df_asw_new = df_asw[keep_cols].copy()

# Rename the ward column to match your other files
df_asw_new.rename(columns={id_col: 'ward_code'}, inplace=True)

In [101]:
# Preview the result
df_asw_new.head(2)

,ward_code,Age_0to4,Age_5to7,Age_8to9,Age_10to14,Age_15,Age_16to17,Age_18to19,Age_20to24,Age_25to29,Age_30to44,Age_45to59,Age_60to64,Age_65to74,Age_75to84,Age_85to89,Age_90andover
0,E05001553 : Bank Top,6.200397,2.876984,1.562500,5.431548,1.041667,2.455357,2.678571,6.994048,8.382937,18.998016,19.618056,6.101190,8.010913,6.324405,1.760913,1.562500
1,E05001554 : Central,6.115984,2.704678,1.461988,4.995127,0.974659,2.217349,3.898635,9.722222,8.771930,20.516569,17.714425,5.068226,8.187135,5.750487,1.267057,0.633528


### Merge MR1, MR2, and ASW

In [102]:
df_mr1.head(2)

,WardCode,WardName,CountingArea,Remain,Leave,Postals,Remain%,Leave%,Unnamed: 8
0,S13002486,Airyhall/Broomhill/Garthdee,Aberdeen City,NaN,NaN,NaN,NaN,NaN,NaN
1,S13002477,Bridge of Don,Aberdeen City,NaN,NaN,NaN,NaN,NaN,NaN


In [103]:
# 1. Create a copy to avoid SettingWithCopy warnings
df_mr1_clean = df_mr1.copy()

# 2. Fill BOTH columns with -1 to handle missing data and allow integer conversion
cols_to_fill = ['Remain', 'Leave']
df_mr1_clean[cols_to_fill] = df_mr1_clean[cols_to_fill].fillna(-1)

# 3. Select and rename columns
df_mr1_clean = df_mr1_clean[['WardCode', 'WardName', 'Leave', 'Remain', 'Postals']]
df_mr1_clean.rename(columns={'WardCode': 'ward_code', 'WardName': 'ward_name'}, inplace=True)

# 4. Convert to integers (Now safe because NaNs are gone)
df_mr1_clean['Remain'] = df_mr1_clean['Remain'].astype(int)
df_mr1_clean['Leave'] = df_mr1_clean['Leave'].astype(int)

# 5. Calculate NVotes
# Rows with missing data will result in NVotes = -2 (-1 + -1)
df_mr1_clean['NVotes'] = df_mr1_clean['Leave'] + df_mr1_clean['Remain']

# 6. Final selection
df_mr1_final = df_mr1_clean[['ward_code', 'ward_name', 'Leave', 'NVotes', 'Postals']]

print(f"Cleaned MR1 has {len(df_mr1_final)} rows.")

Cleaned MR1 has 9602 rows.


In [104]:
# # Filter MR1 to remove empty rows (original file has 9000+ rows, but only 1283 contain vote data)
# df_mr1 = df_mr1.dropna(subset=['Remain']).copy()

# df_mr1_clean = df_mr1[['WardCode', 'WardName', 'Leave', 'Remain', 'Postals']].copy()
# df_mr1_clean.rename(columns={'WardCode': 'ward_code', 'WardName': 'ward_name'}, inplace=True)

# # Ensure vote columns are integers
# df_mr1_clean['Remain'] = df_mr1_clean['Remain'].astype(int)
# df_mr1_clean['Leave'] = df_mr1_clean['Leave'].astype(int)

# # CALCULATE NVOTES
# # Instruction: "NVotes column giving the total number of ‘Leave’ and ‘Remain’ votes"
# df_mr1_clean['NVotes'] = df_mr1_clean['Leave'] + df_mr1['Remain']

# df_mr1_final = df_mr1_clean[['ward_code', 'ward_name', 'Leave', 'NVotes', 'Postals']]

# print(f"Cleaned MR1 has {len(df_mr1_final)} rows.")

In [105]:
df_mr1_final.head(2)

,ward_code,ward_name,Leave,NVotes,Postals
0,S13002486,Airyhall/Broomhill/Garthdee,-1,-2,NaN
1,S13002477,Bridge of Don,-1,-2,NaN


In [106]:
df_mr2.head(2)

,ward_code,White,Black,Asian,Unemp,UnempRate_EA,Students,Owned,OwnedOutright,SocialRent,...,RoutineOccup,LTU,NoQuals,L1Quals,L4Quals_plus,C1C2DE,C2DE,DE,Households,Residents
0,E05000001 : Aldersgate,84.505119,0.750853,8.737201,78.951679,2.784603,3.685504,60.844749,40.525114,4.680365,...,0.655201,1.719902,1.851852,2.592593,77.555556,27.917282,3.692762,1.772526,876,"1,465"
1,E05000005 : Bishopsgate,69.369369,1.801802,21.621622,84.186047,1.860465,6.511628,17.730496,11.347518,2.836879,...,0.930233,1.860465,4.166667,3.240741,65.277778,46.762590,11.510791,5.035971,141,222


In [107]:
df_asw_new.head(2)
print("Shape of dataframe:", df_asw_new.shape)

Shape of dataframe: (8574, 17)


##### Clean `ward_code` for both `df_mr2` and `df_asw_new`

In [108]:
# clean ward_code for df_mr2 and df_asw_new
def clean_ward_code(df, col_name):
    """
    Takes a dataframe and a column name.
    Splits the string at ' :' to keep only the code part.
    Removes any surrounding whitespace.
    """
    # Ensure it's a string, split at the colon, take the first part, and strip spaces
    df[col_name] = df[col_name].astype(str).str.split(':').str[0].str.strip()
    return df

In [109]:
df_asw_clean = clean_ward_code(df_asw_new, 'ward_code').copy()
df_mr2_clean = clean_ward_code(df_mr2, 'ward_code').copy()

In [110]:
df_asw_clean.head(3)
print("Shape of dataframe:", df_asw_clean.shape)

Shape of dataframe: (8574, 17)


In [111]:
df_mr2_clean.head(2)
print("Shape of dataframe:", df_mr2_clean.shape)

Shape of dataframe: (8575, 24)


In [112]:
# We use 'inner' join. This keeps ONLY wards present in ALL files.
# This handles the exclusion of wards with changed boundaries automatically.

# First merge MR1 and MR2
df_merged = pd.merge(df_mr1_final, df_mr2_clean, on='ward_code', how='inner')
df_merged.head(2)

,ward_code,ward_name,Leave,NVotes,Postals,White,Black,Asian,Unemp,UnempRate_EA,...,RoutineOccup,LTU,NoQuals,L1Quals,L4Quals_plus,C1C2DE,C2DE,DE,Households,Residents
0,E05007562,Buckingham,-1,-2,NaN,97.193811,0.131130,1.442434,73.211139,1.550934,...,6.168488,1.057455,18.716745,13.990610,28.794992,72.054795,33.059361,13.333333,"1,644","3,813"
1,E05007563,Churchill,-1,-2,NaN,96.803446,0.430741,1.042847,67.135843,3.600655,...,11.849427,3.371522,30.250741,18.333783,16.823942,88.263666,56.189711,29.662379,"2,069","4,411"


In [113]:
df_merged.columns.to_list()

['ward_code',
 'ward_name',
 'Leave',
 'NVotes',
 'Postals',
 'White',
 'Black',
 'Asian',
 'Unemp',
 'UnempRate_EA',
 'Students',
 'Owned',
 'OwnedOutright',
 'SocialRent',
 'PrivateRent',
 'Deprived',
 'MultiDepriv',
 'HigherOccup',
 'RoutineOccup',
 'LTU',
 'NoQuals',
 'L1Quals',
 'L4Quals_plus',
 'C1C2DE',
 'C2DE',
 'DE',
 'Households',
 'Residents']

In [114]:
# Then merge the result with ASW
merged_df = pd.merge(df_merged, df_asw_clean, on='ward_code', how='inner')

In [115]:
merged_df.columns.to_list()

['ward_code',
 'ward_name',
 'Leave',
 'NVotes',
 'Postals',
 'White',
 'Black',
 'Asian',
 'Unemp',
 'UnempRate_EA',
 'Students',
 'Owned',
 'OwnedOutright',
 'SocialRent',
 'PrivateRent',
 'Deprived',
 'MultiDepriv',
 'HigherOccup',
 'RoutineOccup',
 'LTU',
 'NoQuals',
 'L1Quals',
 'L4Quals_plus',
 'C1C2DE',
 'C2DE',
 'DE',
 'Households',
 'Residents',
 'Age_0to4',
 'Age_5to7',
 'Age_8to9',
 'Age_10to14',
 'Age_15',
 'Age_16to17',
 'Age_18to19',
 'Age_20to24',
 'Age_25to29',
 'Age_30to44',
 'Age_45to59',
 'Age_60to64',
 'Age_65to74',
 'Age_75to84',
 'Age_85to89',
 'Age_90andover']

In [116]:
df_merged.head(2)

,ward_code,ward_name,Leave,NVotes,Postals,White,Black,Asian,Unemp,UnempRate_EA,...,RoutineOccup,LTU,NoQuals,L1Quals,L4Quals_plus,C1C2DE,C2DE,DE,Households,Residents
0,E05007562,Buckingham,-1,-2,NaN,97.193811,0.131130,1.442434,73.211139,1.550934,...,6.168488,1.057455,18.716745,13.990610,28.794992,72.054795,33.059361,13.333333,"1,644","3,813"
1,E05007563,Churchill,-1,-2,NaN,96.803446,0.430741,1.042847,67.135843,3.600655,...,11.849427,3.371522,30.250741,18.333783,16.823942,88.263666,56.189711,29.662379,"2,069","4,411"


In [117]:
merged_df.head(2)

,ward_code,ward_name,Leave,NVotes,Postals,White,Black,Asian,Unemp,UnempRate_EA,...,Age_18to19,Age_20to24,Age_25to29,Age_30to44,Age_45to59,Age_60to64,Age_65to74,Age_75to84,Age_85to89,Age_90andover
0,E05007562,Buckingham,-1,-2,NaN,97.193811,0.131130,1.442434,73.211139,1.550934,...,1.757147,3.619197,4.274849,18.777865,22.606871,8.575924,12.640965,6.818778,1.757147,0.813008
1,E05007563,Churchill,-1,-2,NaN,96.803446,0.430741,1.042847,67.135843,3.600655,...,1.836318,5.690320,5.236908,16.549535,18.249830,6.891861,12.672863,10.292451,3.128542,1.405577


In [118]:
print(f"Original MR1 wards: {len(df_mr1)}")
print(f"Final merged wards: {len(merged_df)}")
print("\nFirst 5 rows of the merged dataset:")
print(merged_df[['ward_code', 'ward_name', 'Leave', 'NVotes']].head())

Original MR1 wards: 9602
Final merged wards: 6345

First 5 rows of the merged dataset:
   ward_code   ward_name  Leave  NVotes
0  E05007562  Buckingham     -1      -2
1  E05007563   Churchill     -1      -2
2  E05007564     Cokeham     -1      -2
3  E05007565   Eastbrook     -1      -2
4  E05007566    Hillside     -1      -2


### Final Merge with RGC

In [119]:
df_rgc.head(2)

,WD17CD,WD17NM,WD17NMW,LAD17CD,LAD17NM,CTY17CD,CTY17NM,GOR10CD,GOR10NM,CTRY17CD,CTRY17NM,FID
0,E05001678,Newington,NaN,E06000010,"Kingston upon Hull, City of",NaN,NaN,E12000003,Yorkshire and The Humber,E92000001,England,1
1,E05001779,Mickleover,NaN,E06000015,Derby,NaN,NaN,E12000004,East Midlands,E92000001,England,2


In [120]:
print(df_rgc.columns.to_list())

['WD17CD', 'WD17NM', 'WD17NMW', 'LAD17CD', 'LAD17NM', 'CTY17CD', 'CTY17NM', 'GOR10CD', 'GOR10NM', 'CTRY17CD', 'CTRY17NM', 'FID']


In [121]:
df_rgc = df_rgc[['WD17CD', 'GOR10NM', 'LAD17NM']].copy()

In [122]:
df_rgc.rename(columns={
    'WD17CD':  'ward_code',
    'GOR10NM': 'Region',
    'LAD17NM': 'Area_Type'
}, inplace=True)

In [123]:
# 3. REMOVE DUPLICATES (As per instruction)
# "Some ward codes were found to be duplicated... but are identical"
df_lookup_clean = df_rgc.drop_duplicates(subset=['ward_code']).copy()

In [124]:
# 4. MERGE WITH YOUR MAIN DATA
# We use 'left' join to ensure we don't lose any of your 1070 wards
df_complete = pd.merge(merged_df, df_lookup_clean, on='ward_code', how='left')

In [125]:
df_complete.head(2)

,ward_code,ward_name,Leave,NVotes,Postals,White,Black,Asian,Unemp,UnempRate_EA,...,Age_25to29,Age_30to44,Age_45to59,Age_60to64,Age_65to74,Age_75to84,Age_85to89,Age_90andover,Region,Area_Type
0,E05007562,Buckingham,-1,-2,NaN,97.193811,0.131130,1.442434,73.211139,1.550934,...,4.274849,18.777865,22.606871,8.575924,12.640965,6.818778,1.757147,0.813008,South East,Adur
1,E05007563,Churchill,-1,-2,NaN,96.803446,0.430741,1.042847,67.135843,3.600655,...,5.236908,16.549535,18.249830,6.891861,12.672863,10.292451,3.128542,1.405577,South East,Adur


In [126]:
df_complete.columns.to_list()

['ward_code',
 'ward_name',
 'Leave',
 'NVotes',
 'Postals',
 'White',
 'Black',
 'Asian',
 'Unemp',
 'UnempRate_EA',
 'Students',
 'Owned',
 'OwnedOutright',
 'SocialRent',
 'PrivateRent',
 'Deprived',
 'MultiDepriv',
 'HigherOccup',
 'RoutineOccup',
 'LTU',
 'NoQuals',
 'L1Quals',
 'L4Quals_plus',
 'C1C2DE',
 'C2DE',
 'DE',
 'Households',
 'Residents',
 'Age_0to4',
 'Age_5to7',
 'Age_8to9',
 'Age_10to14',
 'Age_15',
 'Age_16to17',
 'Age_18to19',
 'Age_20to24',
 'Age_25to29',
 'Age_30to44',
 'Age_45to59',
 'Age_60to64',
 'Age_65to74',
 'Age_75to84',
 'Age_85to89',
 'Age_90andover',
 'Region',
 'Area_Type']

In [127]:
df_complete.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6345 entries, 0 to 6344
Data columns (total 46 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ward_code      6345 non-null   object 
 1   ward_name      6345 non-null   object 
 2   Leave          6345 non-null   int64  
 3   NVotes         6345 non-null   int64  
 4   Postals        1070 non-null   object 
 5   White          6345 non-null   float64
 6   Black          6345 non-null   float64
 7   Asian          6345 non-null   float64
 8   Unemp          6345 non-null   float64
 9   UnempRate_EA   6345 non-null   float64
 10  Students       6345 non-null   float64
 11  Owned          6345 non-null   float64
 12  OwnedOutright  6345 non-null   float64
 13  SocialRent     6345 non-null   float64
 14  PrivateRent    6345 non-null   float64
 15  Deprived       6345 non-null   float64
 16  MultiDepriv    6345 non-null   float64
 17  HigherOccup    6345 non-null   float64
 18  RoutineO

In [128]:
cols_to_convert = ["Households", "Residents"]

for col in cols_to_convert:
    if col in df_complete.columns:
        df_complete[col] = pd.to_numeric(
            df_complete[col].astype(str).str.replace(',', ''),
            errors = 'coerce'
        )

print(df_complete[cols_to_convert].dtypes)

Households    int64
Residents     int64
dtype: object


In [129]:
df_complete[["Households", "Residents", "Region", "Area_Type"]]

,Households,Residents,Region,Area_Type
0,1644,3813,South East,Adur
1,2069,4411,South East,Adur
2,1896,4198,South East,Adur
3,1883,4607,South East,Adur
4,1778,4221,South East,Adur
...,...,...,...,...
6340,3679,9181,South East,Wycombe
6341,3500,8101,South East,Wycombe
6342,2271,5261,South East,Wycombe
6343,2657,6562,South East,Wycombe


In [132]:
# 1. Define the Adult Age Bands and their Midpoints
# We use the midpoint of each range (e.g., 30 to 44 -> (30+44)/2 = 37)
# For '90andover', we estimate 93 as a standard demographic approximation.
age_midpoints = {
    'Age_18to19': 18.5,
    'Age_20to24': 22.0,
    'Age_25to29': 27.0,
    'Age_30to44': 37.0,
    'Age_45to59': 52.0,
    'Age_60to64': 62.0,
    'Age_65to74': 69.5,
    'Age_75to84': 79.5,
    'Age_85to89': 87.0,
    'Age_90andover': 93.0
}

# 2. Calculate the "Weighted Sum" and "Total Adult Percentage"
weighted_sum = 0
total_adult_pct = 0

for col, midpoint in age_midpoints.items():
    if col in df_complete.columns:
        # Add (Percentage * Midpoint) to the weighted sum
        weighted_sum += df_complete[col] * midpoint
        # Add Percentage to the total adult percentage
        total_adult_pct += df_complete[col]

# 3. Calculate AdultMeanAge
# Formula: Sum(Percentage * Midpoint) / Sum(Percentage)
df_complete['AdultMeanAge'] = weighted_sum / total_adult_pct

# 4. Verification
print("AdultMeanAge created successfully!")
print(df_complete[['ward_code', 'AdultMeanAge', 'Age_45to59']].head())

# Optional: Check correlation to ensure it aligns with expectations
print("\nCorrelation between Calculated Mean Age and Age_45to59 band:")
print(df_complete[['AdultMeanAge', 'Age_45to59']].corr().iloc[0,1])

AdultMeanAge created successfully!
   ward_code  AdultMeanAge  Age_45to59
0  E05007562     52.408448   22.606871
1  E05007563     53.579806   18.249830
2  E05007564     51.918444   22.081944
3  E05007565     48.195447   18.385066
4  E05007566     51.652889   21.606254

Correlation between Calculated Mean Age and Age_45to59 band:
0.719737352657594


In [133]:
df_complete.to_csv('final_dataset_1070_wards.csv', index=False, encoding='utf-8')

print("File saved successfully as 'final_dataset_1070_wards.csv'")

File saved successfully as 'final_dataset_1070_wards.csv'
